In [1]:
import torch
import torch.nn as nn
from torch import optim

import sys

In [2]:
# local imports 
sys.path.append("..")

from transformer_model import generate_text, train_model, tokenize_raw_data
from data import ArticlesIter, articles_filepath

2023-03-29 13:41:45.508940: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 13:41:46.910089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-29 13:41:46.910223: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-29 13:41:46.910234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## Setup

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load Vocab

In [4]:
SAVE_PATH = "../transformer_model/vocab_save/" + "embeddings_vocab.pt"

emb_vocab_obj = torch.load(SAVE_PATH)

print("Vocab size:", len(emb_vocab_obj))

Vocab size: 400004


### Load Model

In [5]:
SAVE_PATH = "../transformer_model/model_save/" + "glove_emb_model.pt"

emb_model = torch.load(SAVE_PATH)

print("Loaded hyperparameters:")
print("---------------------------")
print("Embedding Dimension:", emb_model.embedding_dim)
print("Hidden Dim:         ", emb_model.hidden_dim)
print("Num Layers:         ", emb_model.num_layers)
print("Num Heads:          ", emb_model.num_heads)
print("Max Length:         ", emb_model.max_len)
print("Dropout Per.:       ", emb_model.dropout_p)

Loaded hyperparameters:
---------------------------
Embedding Dimension: 300
Hidden Dim:          300
Num Layers:          2
Num Heads:           4
Max Length:          100
Dropout Per.:        0.2


## Test Text Generation
Given our mappings we just read in we can now have the transformer make some text.

In [6]:
# start_seq = ["<sos>", "the", "quick", "brown", "fox"]
# number_of_words = 10

# for word in start_seq:
#     if word != "<sos>":
#         print(word, end=" ")
        
# for word in generate_text(
#     model=emb_model,
#     vocab=emb_vocab_obj,
#     start_seq=start_seq,
#     max_length=number_of_words):
#     print(word, end=" ")

## Train the Model

In [7]:
learning_rate = 3e-4

optimizer = optim.Adam(emb_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

The data will be split into 1000 batches. For each batch, the content will be pulled out. This content will be tokenized and fed into the *train_model* function.

In [8]:
article_iter = ArticlesIter(batch_size=1000)
total_loss = 0.0
total_batches = 0

for batch_i, batch in enumerate(article_iter):
    print("Training batch:", batch_i)
    batch_content = batch["Content"]

    for article in batch_content:
        tokenized_batch_content = tokenize_raw_data(article, emb_vocab_obj)
        print(tokenized_batch_content)

        # total_loss += train_model(
        #     model=emb_model,
        #     vocab=emb_vocab_obj,
        #     optimizer=optimizer,
        #     criterion=criterion,
        #     train_data=batch_content,
        #     device=device
        # )
    total_batches += 1

print("Average Loss:", total_loss/total_batches)


Training batch: 0


/home/cstainsby/.local/lib/python3.8/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


KeyError: 'N'